# Implementación de algoritmo _CVRP_ en simulación generada

In [20]:
import pandas as pd
import numpy as np
from geopy.distance import geodesic
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
from __future__ import print_function

### Importar datos de simulación

In [21]:
simulacion_clientes = pd.read_csv("simulacion_clientes1.csv")
simulacion_clientes.head()

,ciudad,Colonia,codigo postal,Calle,numero de casa,Aleatorio,Numero de pedidos,23,29,30,...,52940,52942,52984,52993,53014,53475,Volumen por pedidos (m^3),Coordenadas,Latitud,Longitud
0,Monterrey,FIERRO,64590,JESUS M GARZA,4026.0,0.527082,1,0,0,0,...,0,0,0,0,0,0,0.017079,"(25.68654935, -100.2772068)",25.686549,-100.277207
1,Monterrey,CIUDAD NATURA,64989,AV ABRAHAM LIN,4001.0,0.748310,2,0,0,0,...,0,0,0,0,0,0,0.290464,"(25.763761, -100.408254)",25.763761,-100.408254
2,Monterrey,FOME 6 F S T D M,64233,CALLE JOSE ANTO,7319.0,0.869673,2,0,0,0,...,0,0,0,0,0,0,0.001022,"(25.79072633, -100.40731977)",25.790726,-100.407320
3,Monterrey,PORTAL DEL FRAILE,66064,FRAY JOSE,729.0,0.628113,1,0,0,0,...,0,0,0,0,0,0,0.214957,"(25.8003649, -100.4111165)",25.800365,-100.411117
4,Monterrey,CUMBRES 3 SEC,64610,HACIENDA DEL CA,310.0,0.485124,1,0,0,0,...,0,0,0,0,0,0,0.001254,"(25.743958, -100.406852)",25.743958,-100.406852


In [22]:
nuevas_coordenadas = []
for i in range(simulacion_clientes.shape[0]):
    coord =  (simulacion_clientes.iloc[i,4770],simulacion_clientes.iloc[i,4771])
    nuevas_coordenadas.append(coord)

simulacion_clientes["Coordenadas"] = nuevas_coordenadas

In [23]:
simulacion_clientes["Coordenadas"][0]

(25.68654935, -100.2772068)

In [24]:
#Ejemplo
distancia = str(geodesic(simulacion_clientes.iloc[0,4769], simulacion_clientes.iloc[1,4769]).km)
print("Distancia entre dos puntos",distancia)

Distancia entre dos puntos 15.687694259452297


In [25]:
def generar_distancias(clientes):
    matriz_distancias = []

    for i in range(clientes.shape[0]):
        coord1 = clientes.iloc[i,4769]
        distancias =  []

        for j in range(clientes.shape[0]):
            if j != i:
                coord2 = clientes.iloc[j,4769]
                distancia = float(geodesic(coord1, coord2).km)
                distancias.append(distancia)
            else:
                distancias.append(0)

        matriz_distancias.append(distancias)


    return matriz_distancias


In [26]:
distancias = generar_distancias(simulacion_clientes)

In [27]:
pd.DataFrame(np.matrix(distancias)).head()

,0,1,2,3,4,5,6,7,8,9,...,213,214,215,216,217,218,219,220,221,222
0,0.000000,15.687694,17.425235,18.425636,14.482149,4.158712,4.453844,2.706486,8.884974,11.780051,...,21.246863,7.883956,17.685850,14.321462,14.321462,11.113727,16.758784,13.579075,19.859042,13.226262
1,15.687694,0.000000,2.988812,4.065302,2.198365,14.694221,17.847716,12.989704,6.884350,4.285299,...,5.581091,9.767339,4.692945,9.935490,9.935490,14.371188,1.867252,7.075472,9.871990,10.116695
2,17.425235,2.988812,0.000000,1.133668,5.181416,16.987334,20.059013,14.727781,9.031922,6.890901,...,5.081594,10.553959,1.919906,8.719990,8.719990,14.062171,1.122377,6.227439,7.182484,9.225395
3,18.425636,4.065302,1.133668,0.000000,6.263635,18.084174,21.140667,15.736571,10.128493,8.024394,...,4.919663,11.379765,1.600512,8.912715,8.912715,14.516427,2.210169,6.662113,6.547056,9.540049
4,14.482149,2.198365,5.181416,6.263635,0.000000,13.033233,16.223973,11.838029,5.609286,2.703847,...,6.966670,9.539921,6.809041,11.098107,11.098107,14.780028,4.062602,8.191045,11.869299,11.069079


In [28]:
demandas = simulacion_clientes["Volumen por pedidos (m^3)"].tolist()

In [29]:
# El primer nodo no tiene demanda
demandas[0] = 0

In [30]:
# Suma de la demanda de todos los clientes(en m^3)
simulacion_clientes["Volumen por pedidos (m^3)"].sum()

50.095983700000005

In [31]:
pedidos = simulacion_clientes["Numero de pedidos"].to_list()

In [32]:
distancias_escaladas = [[int(round(d * 1000)) for d in row] for row in distancias] 

In [33]:
demandas_escaladas = [int(round(p * 1000)) for p in demandas]

In [34]:
capacidades_escaladas = [c * 1000 for c in [8,8,8,8,8,8,8]]
capacidades_escaladas

[8000, 8000, 8000, 8000, 8000, 8000, 8000]

### Algoritmo de busqueda de rutas (_CVRP_)

In [35]:
def create_data_model():
    """Stores the data for the problem."""
    data = {}
    data["distance_matrix"] = distancias_escaladas
    data["demands"] = demandas_escaladas
    data["vehicle_capacities"] = capacidades_escaladas
    data["num_vehicles"] = len(capacidades_escaladas)
    data["depot"] = 0         
    
    return data

In [36]:


def print_solution(data, manager, routing, solution):
    """Prints solution on console."""
    print(f"Objective: {solution.ObjectiveValue()}")
    total_distance = 0
    total_load = 0
    for vehicle_id in range(data["num_vehicles"]):
        index = routing.Start(vehicle_id)
        plan_output = f"Route for vehicle {vehicle_id}:\n"
        route_distance = 0
        route_load = 0
        while not routing.IsEnd(index):
            node_index = manager.IndexToNode(index)
            route_load += data["demands"][node_index]
            plan_output += f" {node_index} Load({route_load/ 1000}) -> "
            previous_index = index
            index = solution.Value(routing.NextVar(index))
            route_distance += routing.GetArcCostForVehicle(
                previous_index, index, vehicle_id
            )
        plan_output += f" {manager.IndexToNode(index)} Load({route_load / 1000})\n"
        plan_output += f"Distance of the route: {route_distance / 1000}m\n"
        plan_output += f"Load of the route: {route_load / 1000}\n"
        print(plan_output)
        total_distance += route_distance
        total_load += route_load
    print(f"Total distance of all routes: {total_distance / 1000}m")
    print(f"Total load of all routes: {total_load / 1000}")


def main():
    """Solve the CVRP problem."""
    # Instantiate the data problem.
    data = create_data_model()

    # Create the routing index manager.
    manager = pywrapcp.RoutingIndexManager(
        len(data["distance_matrix"]), data["num_vehicles"], data["depot"]
    )

    # Create Routing Model.
    routing = pywrapcp.RoutingModel(manager)

    # Create and register a transit callback.
    def distance_callback(from_index, to_index):
        """Returns the distance between the two nodes."""
        # Convert from routing variable Index to distance matrix NodeIndex.
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data["distance_matrix"][from_node][to_node]

    transit_callback_index = routing.RegisterTransitCallback(distance_callback)

    # Define cost of each arc.
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

    # Add Capacity constraint.
    def demand_callback(from_index):
        """Returns the demand of the node."""
        # Convert from routing variable Index to demands NodeIndex.
        from_node = manager.IndexToNode(from_index)
        return data["demands"][from_node]

    demand_callback_index = routing.RegisterUnaryTransitCallback(demand_callback)
    
    routing.AddDimensionWithVehicleCapacity(
        demand_callback_index,
        0,  # null capacity slack
        data["vehicle_capacities"],  # vehicle maximum capacities
        True,  # start cumul to zero
        "Capacity",
    )

    # Setting first solution heuristic.
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.local_search_metaheuristic = (
        routing_enums_pb2.LocalSearchMetaheuristic.GUIDED_LOCAL_SEARCH
    )
    search_parameters.time_limit.FromSeconds(1)

    # Solve the problem.
    solution = routing.SolveWithParameters(search_parameters)

    # Print solution on console.
    if solution:
        print_solution(data, manager, routing, solution)


if __name__ == "__main__":
    main()

Objective: 456581
Route for vehicle 0:
 0 Load(0.0) ->  65 Load(0.001) ->  6 Load(0.91) ->  22 Load(0.914) ->  79 Load(1.088) ->  30 Load(1.089) ->  15 Load(1.304) ->  42 Load(2.198) ->  28 Load(2.258) ->  41 Load(2.263) ->  11 Load(3.726) ->  71 Load(4.123) ->  0 Load(4.123)
Distance of the route: 89.491m
Load of the route: 4.123

Route for vehicle 1:
 0 Load(0.0) ->  43 Load(0.001) ->  80 Load(0.004) ->  40 Load(0.041) ->  78 Load(0.047) ->  49 Load(0.047) ->  25 Load(0.048) ->  13 Load(0.655) ->  47 Load(0.666) ->  17 Load(0.672) ->  37 Load(0.694) ->  68 Load(0.696) ->  19 Load(0.911) ->  60 Load(0.912) ->  208 Load(1.44) ->  212 Load(1.51) ->  205 Load(1.607) ->  74 Load(2.144) ->  3 Load(2.359) ->  59 Load(3.288) ->  69 Load(3.292) ->  2 Load(3.293) ->  67 Load(3.294) ->  210 Load(3.318) ->  219 Load(3.319) ->  197 Load(3.878) ->  81 Load(3.987) ->  1 Load(4.277) ->  23 Load(4.28) ->  4 Load(4.281) ->  64 Load(4.583) ->  58 Load(6.579) ->  213 Load(6.589) ->  184 Load(6.59) ->  5